<a href="https://colab.research.google.com/github/nullpitch-dev/DS_L1_Notebooks/blob/master/DS_L1_EX_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# [0]

import pandas as pd

# Loading data
url = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/13.csv'
data = pd.read_csv(url)

In [0]:
# [0]

# Preprocessing data
data = data[data['Year'].isna() == False]
data = data.sort_values(by=['Name', 'Platform', 'Year'],
                        ascending=[True, True, True])
data = data.drop_duplicates(subset=['Name', 'Platform'], keep='last')
data[{'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales'}].fillna(0)
data['Global_sales'] = data[{'NA_Sales', 'EU_Sales',
                             'JP_Sales', 'Other_Sales'}].sum(axis=1)
platforms = data.groupby('Platform').sum()['Global_sales']
platforms = platforms[platforms.values >= 20000]
data = data[data['Platform'].isin(platforms.index)]
data = data.reset_index().iloc[:, 1:]

In [0]:
# [1]

from sklearn.neighbors import KNeighborsClassifier

type_max = data['Global_sales'].max()
type_min = data['Global_sales'].min()
type_median = data['Global_sales'].median()

def checkType(x):
    if x == type_max or x == type_min:
        return 'Type1'
    elif x == type_median:
        return 'Type2'
    else:
        return 'NotCheckPoint'


data['Check_point'] = data['Global_sales'].apply(lambda x: checkType(x))

train_x = data[data['Check_point'] != 'NotCheckPoint'][{'NA_Sales', 'EU_Sales',
                                                        'JP_Sales',
                                                        'Other_Sales'}]
train_y = data[data['Check_point'] != 'NotCheckPoint']['Check_point']

knn = KNeighborsClassifier(n_neighbors=1, p=2, metric='minkowski')
knn_model = knn.fit(train_x, train_y)

test_x = data[{'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales'}]
pred = knn_model.predict(test_x)

results = pd.DataFrame({'pred': pred})

data_pro = pd.concat([data, results], axis=1)
data_pro = data_pro[
         (data_pro['Check_point'] != 'Type1') & (data_pro['pred'] != 'Type1')]

global_sales_mean = data_pro['Global_sales'].mean()
print(f'Answer [1] : Mean of Global_sales = {global_sales_mean:.3f}')

Answer [1] : Mean of Global_sales = 74.201


In [0]:
# [2]

data_temp = data_pro.groupby(['Name', 'Genre']).agg({'Platform': 'count'})
data_temp = data_temp.groupby('Genre').mean()
data_temp = data_temp.sort_values(by='Platform', ascending=False)

top_genre = data_temp.index[0]
bottom_genre = data_temp.index[-1]

print(f'Answer [2] : Top genre = {top_genre}, Bottom genre = {bottom_genre}')

Answer [2] : Top genre = Action, Bottom genre = Puzzle


In [0]:
# [3]

from scipy import stats

data_temp = pd.merge(left=data_pro[data_pro['Platform'] == 'PC'],
                   right=data_pro[data_pro['Platform'] == 'X360'],
                   on='Name')
data3 = data_pro[data_pro['Name'].isin(data_temp['Name']) == True]

data3_pc = data3[data3['Platform'] == 'PC']['Global_sales']
data3_x360 = data3[data3['Platform'] == 'X360']['Global_sales']

t_value, p_value = stats.ttest_rel(data3_pc, data3_x360)

print(f'Answer [3] : t-value = {abs(t_value):.3f}')

Answer [3] : t-value = 7.772


In [0]:
# [4]

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

data4 = pd.get_dummies(data_pro, columns=['Platform'], drop_first=True)
data4['Sports'] = data4['Genre'].apply(lambda x: 0 if x != 'Sports' else 1)

cols = list(data4.columns)[4:8] + list(data4.columns)[12:-1]

train_x = data4[data4['XGRP'] == 'A'][cols]
train_y = data4[data4['XGRP'] == 'A']['Sports']
test_x = data4[data4['XGRP'] == 'B'][cols]
test_y = data4[data4['XGRP'] == 'B']['Sports']

lr = LogisticRegression(C=100000, random_state=1234, penalty='l2',
                        solver='newton-cg')
lr_model = lr.fit(train_x, train_y)
pred = lr_model.predict_proba(test_x)

odds = np.matrix(pred[:, 1] / pred[:, 0])
pred = np.hstack((pred, odds.T))

pred_df = pd.DataFrame(pred, columns=['No', 'Yes', 'Odds'])
pred_df['Pred'] = pred_df['Odds'].apply(lambda x: 1 if x >= 0.12 else 0)

print(f"Answer [4] : Accuracy = {accuracy_score(test_y, pred_df['Pred']):.3f}")

Answer [4] : Accuracy = 0.383
